In [1]:
import torch
import torch.nn as nn #all NN modules are in this, loss functns also
import torch.optim as optim #all optimaizatn algos like adam sgd in this
import torch.nn. functional as F #it have activatn fucntions like relu n all, nn also have them
from torch.utils.data import DataLoader #easier dataset management
import torchvision.datasets as datasets #for getting data from torch
import torchvision.transforms as transforms #transformatns that we can perform on our dataset

In [2]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super().__init__()
        self.conv1= nn.Conv2d(in_channels= in_channels, out_channels= 8, kernel_size= (3, 3), stride= (1, 1), padding= (1, 1))
        self.pool= nn.MaxPool2d(kernel_size= (2, 2), stride= (2, 2))
        self.conv2= nn.Conv2d(in_channels= 8, out_channels= 16, kernel_size= (3, 3), stride= (1, 1), padding= (1, 1))
        self.fc1= nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x= F.relu(self.conv1(x))
        x= self.pool(x)
        x= F.relu(self.conv2(x))
        x= self.pool(x)
        x= x.reshape(x.shape[0], -1)
        x= self.fc1(x)
        return x

In [3]:
model= CNN()
x= torch.randn(64, 1, 28, 28)
model(x).shape

torch.Size([64, 10])

In [4]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 0.01
batch_size = 128
num_epochs = 5

In [6]:
# Load Data
train_dataset = datasets.MNIST(root='data/', train=True, transform=transforms.ToTensor (), download=False)
#data come in dataset/ and it is in numpy so transorfm change to tensor and if not in root then download
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


test_dataset = datasets.MNIST(root='data/', train=False, transform=transforms.ToTensor (), download=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = CNN(in_channels= in_channels, num_classes=num_classes).to(device)

In [9]:
print(model)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)


In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss ()
optimizer = optim.Adam(model.parameters (), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
  print(f"epoch {epoch+1}/{num_epochs}")
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)
    scores = model(data)
    loss = criterion(scores, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer .step()

epoch 1/5
epoch 2/5
epoch 3/5
epoch 4/5
epoch 5/5


In [ ]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model) :
  if loader.dataset.train:
    print ('Checking accuracy on training data')
  else:
    print ('Checking accuracy on test data')
  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x. to(device=device)
      y = y.to(device=device)
      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y) .sum()
      num_samples += predictions.size (0)

    print (f'Got {num_correct} / {num_samples} with accuracy {float (num_correct)/float (num_samples) *100: .2f} ')
  model. train()

In [16]:
check_accuracy (train_loader, model)
check_accuracy (test_loader, model)

Checking accuracy on training data
Got 59377 / 60000 with accuracy  98.96 
Checking accuracy on test data
Got 9879 / 10000 with accuracy  98.79 
